# Tools for Experiment 1

## Graphstate on Ring

In [ ]:
from qiskit.circuit.library import GraphState

In [ ]:
def graphstate_manhatten_ring(ring_size=12):
    def ring_map(size):
        for j in range(size):
            yield [j, (j+1)%ring_size]
            yield [(j+1)%ring_size, j]

    coupling_map = list(ring_map(ring_size))

    rows, cols = zip(*coupling_map)

    matrix = numpy.zeros((ring_size, ring_size))
    matrix[rows, cols] = 1

    qc = GraphState(matrix)
    qc.measure_all()

    return qc

## Evaluate

In [ ]:
from qiskit import assemble
from qiskit.transpiler import CouplingMap
from qiskit.transpiler.passes import CSPLayout, CspRbsLayout, DenseLayout, \
                                     NoiseAdaptiveLayout, SabreLayout
from qiskit.providers.aer import QasmSimulator
from qiskit.providers.aer.noise import NoiseModel

In [ ]:
def exp1_evaluate(circuit, layout_method, backend, ideal=True, shots=1, seed=None):
    """
    layout_method:
      - csplayout:
    """
    coupling_map = CouplingMap(backend.configuration().coupling_map)
    backend_properties = backend.properties()

    if layout_method == 'csplayout':
        layout = CSPLayout(coupling_map, seed=seed, call_limit=None, time_limit=None)
    elif layout_method == 'csprbslayout':
        layout = CspRbsLayout(coupling_map, seed=seed, call_limit=None, time_limit=None,
                              solution_limit=20, iteration_limit=10, backend_properties=backend_properties)
    elif layout_method == 'dense':
        layout = DenseLayout(coupling_map, None if ideal else backend_properties)
    elif layout_method == 'noise_adaptive':
        layout = NoiseAdaptiveLayout(backend_properties)
    elif layout_method == 'sabre':
        layout = SabreLayout(coupling_map, max_iterations=1, seed=seed)
    else:
        raise Exception('layout_method unknown %s' % layout_method)

    if ideal:
        simulator = QasmSimulator(configuration=backend.configuration(),
                                  method='stabilizer')
    else:
        noise_model = NoiseModel.from_backend(backend, thermal_relaxation=False)
        simulator = QasmSimulator(configuration=backend.configuration(),
                                  noise_model=noise_model,
                                  method='stabilizer')

    passmanager = custom_pass_manager(backend, layout, seed=seed)

    times = {}
    count_ops_after_map = {}

    def callback(**kwargs):
        times[kwargs['pass_'].__class__.__name__] = kwargs['time']
        if kwargs['pass_'].__class__.__name__ == 'StochasticSwap':
            count_ops_after_map.update(kwargs['dag'].count_ops())

    transpiled = passmanager.run(circuit, callback=callback)
    needed_swaps = 0 if passmanager.property_set['is_swap_mapped'] else count_ops_after_map['swap']

    qobj = assemble(transpiled, backend, shots=shots, seed_simulator=seed)
    result = simulator.run(qobj).result()
    counts = result.get_counts()

    return counts, times[layout.__class__.__name__], needed_swaps

## Write to Dataframe and file

In [ ]:
def write_to_dataframe(total_result):
    columns = ["seed", "state", "ideal_count", "noise_count", \
               "ideal_time", "noise_time", "ideal_swaps_needed", "noise_swaps_needed"]
    df = pd.DataFrame(columns=columns)
    
    ideal_counts = total_result["ideal_counts"]
    noise_counts = total_result["noise_counts"]
    states = set(ideal_counts.keys()) | set(noise_counts.keys())
    for state in states:
        df.loc[len(df)] = [total_result["seed"], state, ideal_counts.get(state, 0), noise_counts.get(state, 0), \
                           total_result["ideal_time"], total_result["noise_time"], total_result["ideal_swaps_needed"], \
                           total_result["noise_swaps_needed"]]

    return df

## Custom Pass Manager

In [ ]:
from qiskit.transpiler import PassManager, CouplingMap
from qiskit.transpiler.passes import BasisTranslator
from qiskit.transpiler.passes import UnrollCustomDefinitions
from qiskit.transpiler.passes import Unroll3qOrMore
from qiskit.transpiler.passes import CheckMap
from qiskit.transpiler.passes import BarrierBeforeFinalMeasurements
from qiskit.transpiler.passes import StochasticSwap
from qiskit.transpiler.passes import FullAncillaAllocation
from qiskit.transpiler.passes import EnlargeWithAncilla
from qiskit.transpiler.passes import ApplyLayout
from qiskit.circuit.equivalence_library import SessionEquivalenceLibrary as sel

In [ ]:
def custom_pass_manager(backend, layout_instance, seed=None):
    basis_gates = backend.configuration().basis_gates
    coupling_map = CouplingMap(backend.configuration().coupling_map)

    def _swap_condition(property_set):
        return not property_set['is_swap_mapped']

    _embed = [FullAncillaAllocation(coupling_map), EnlargeWithAncilla(), ApplyLayout()]
    _unroll3q = Unroll3qOrMore()
    _swap_check = CheckMap(coupling_map)
    _swap = [BarrierBeforeFinalMeasurements()]
    _swap += [StochasticSwap(coupling_map, trials=20, seed=seed)]
    _unroll = [UnrollCustomDefinitions(sel, basis_gates), BasisTranslator(sel, basis_gates)]

    pm = PassManager()
    pm.append(layout_instance)
    pm.append(_embed)
    pm.append(_unroll3q)
    pm.append(_swap_check)
    pm.append(_swap, condition=_swap_condition)
    pm.append(_unroll)
    return pm